In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
import pickle as pk
# import matplotlib.pyplot as plt

In [2]:
pixels = 100
dimensions = (pixels, pixels)
bins = 2
mid_layer_1 = 1000
num_epochs = 3
learning_rate = 0.001
reduction_factor = 0.8

In [3]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

In [4]:
train_images = []
train_labels = np.array([])

folder_path = "/mnt/c/NN/Lukemia/C-NMC_Leukemia/training_data/fold_0/"

for f in listdir(folder_path + "all"):
    im = img.open(folder_path + "all/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 1)

for f in listdir(folder_path + "hem"):
    im = img.open(folder_path + "hem/" + f)
    im = im.resize(dimensions)
    im = im.convert('L')
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 0)

In [5]:
train_images = np.array(train_images)
print(np.shape(train_images))
print(np.shape(train_labels))

(10661, 100, 100)
(10661,)


In [6]:
randomize = np.arange(np.size(train_labels, 0))
np.random.shuffle(randomize)
train_images = train_images[randomize]
train_labels = train_labels[randomize]

test_images = train_images[int(0.8 * np.size(train_labels)):, :, :]
test_labels = train_labels[int(0.8 * np.size(train_labels)):]

train_images = train_images[:int(0.8 * np.size(train_labels)), :, :]
train_labels = train_labels[:int(0.8 * np.size(train_labels))]

train_images = np.squeeze(train_images)
test_images = np.squeeze(test_images)
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

In [7]:
print(dimensions)

(100, 100)


In [8]:
model = models.Sequential()
model.add(layers.Conv2D(pixels, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(mid_layer_1, activation='relu'))
model.add(layers.Dense(bins))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
history = model.fit(train_images, train_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_images, test_labels))

Epoch 1/3
267/267 [==============================] - ETA: 0s - loss: 3.4027 - accuracy: 0.7906
Epoch: 0. Reducing Learning Rate from 0.0010000000474974513 to 0.000800000037997961
267/267 [==============================] - 277s 1s/step - loss: 3.4027 - accuracy: 0.7906 - val_loss: 0.4683 - val_accuracy: 0.8050
Epoch 2/3
267/267 [==============================] - ETA: 0s - loss: 0.4219 - accuracy: 0.8158
Epoch: 1. Reducing Learning Rate from 0.000800000037997961 to 0.000640000042039901
267/267 [==============================] - 295s 1s/step - loss: 0.4219 - accuracy: 0.8158 - val_loss: 0.4626 - val_accuracy: 0.8036
Epoch 3/3
267/267 [==============================] - ETA: 0s - loss: 0.3858 - accuracy: 0.8296
Epoch: 2. Reducing Learning Rate from 0.000640000042039901 to 0.0005120000569149852
267/267 [==============================] - 300s 1s/step - loss: 0.3858 - accuracy: 0.8296 - val_loss: 0.4881 - val_accuracy: 0.7961


In [10]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")

67/67 - 10s - loss: 0.4881 - accuracy: 0.7961 - 10s/epoch - 155ms/step
Accuracy = 79.606 %


In [11]:
filename = 'model.bin'
pk.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://66c52fdf-a02b-4a38-8a8b-e351709ae5e1/assets


In [ ]:
model = pk.load(open(filename, 'rb'))